In [30]:
from numpy import loadtxt
path = loadtxt(r"C:\Users\localadmin.TUD0035314\Documents\GitHub\New_world\robot_trajectory.txt")

# Lets prune the information in our hand since not information is gained from the path in between the collisions
# Only the poses of collision are needed, that define an event based system. The control input should act on the collision pose
# and see if possible if the entire pose trajectory is same as the list of landmarks (even thought they start to fuse in 
# formation of hyperplanes or half-spaces)
poses = []
poses.append(path[0])
for i in range(len(path)):
    if path[i][3]!=0:
#         poses.append(path[i-1])
        poses.append(path[i]) 
#         poses.append(path[i+1])

In [ ]:
from math import tan, atan, sqrt, sec
from numpy import eye
from random import gauss, randint, uniform
import copy

# Creating Particle class, where each particle is an instantiation of this class
class Particle:
    def __init__(self,pose):
        self.pose = pose
        self.landmark_poses = [] # (list of x,y,theta) or (list of x,y,a,b) where aTx<=b where (theta, x or y) in the first 
        # option define the hyperplane aTx<=b
        self.landmark_covariances = [] # list of 3x3 matrices - If we go for a 1D covariance as in GUI, we consider only the 
        # correspoding axis covariances (x or y)
        
    def number_of_landmarks(self):
        # Returns the number of landmarks for a particle and also the number of collisions to define a hyperplane or half-space 
        # in future
        return len(self.landmark_poses)
    
    @staticmenthod
    def motion(pose, control, WB, dt):  # Unicycle motion model
        control[1]=control[1]%360
        angle = ((control[1]+pose[2]*180/pi)%360)*pi/180
        return ([x+y for x,y in zip(pose,[control[0]*dt*cos(angle), 
                                          control[0]*dt*sin(angle), 
                                          control[0]*dt*sin(control[1]*pi/180)/WB])])
    
    def move(self,control,WB,dt):
        # Given the control, move the robot in a direction
        self.pose = self.motion(self.pose,control,WB,dt)
        
    def expected_measurement(pose,landmark): # Pose in the frame of robot and landmark orientation in the world frame 
        # This function is called only if the landmark list is non-empty
        # The position and orientation of the landmark is defined in the world-coordinate frame and not in the robot frame of 
        # orientation.
        theta = pose[2]
        phi = landmark[2]
        
        return tan(phi-theta) # Ratio of the accelerations ax/ay       
    
    def measurement_correspondence(self,pose,measurement,number_of_landmarks,Qt_measurment_covariance):
        # For a given measurment, returns a set of correspondences over the list of available landmarks
        likelihoods = []
        for i in range(number_of_landmarks):
            # Measurement  - Ratio of ax/ay
            likelihoods.append(self.landmark_correspondence(measurement,i,Qt_measurment_covariance))
        return likelihoods
    
    @staticmethod
    def dh_dlandmark(pose,landmark):
        theta = pose[2]
        phi = landmark[2]
        # H = Derivative of self.expected_measurement with respect to the landmark at the point of pose and landmark 
        # H = 1x3 where there is one output and three landmark variables x,y,phi
        H = [0, 0, -sec(theta-phi)**2 ] # Odometry measurement not taken into account
        
        return H
        
    def landmark_correspondence_likelihood(self,measurement,landmark_number,Qt_measurement_covariance):
        # For a given measurment and a landmark number, it returns a suitable likelihood value of the correspondence

        landmark = self.landmark_poses[landmark_number]
        phi = landmark[2]
        theta = self.pose[2]
        
        # Prune the landmark which are not in the robot's frame of collision
        if phi == 90 * floor(theta/90) or phi == 90 * ceil(theta/90): 
            
            zhat = self.expected_measurement(self.pose,landmark)
            
            # Computing the derivative of measurement function with respect to the landmark variables
            H = dh_dlandmark(self.pose,landmark)
            # Computing the variance propagation of landmarks and the measurement
            Q = H * self.landmark_covariances[landmark_number] * H.T + Qt_measurement_covariance
            
            dz = measurement - zhat
            
            # Computing (log) likelihood
            # l = 
            
            # Now the orientations have been checked. Check whether they lie close to each other
            # If l is higher than the threshold, then lets compute the threshold whether they lie close to each other with the 
            # same orientation. Later, we have to incorporate the ones with the corners so that they can form halfspaces
            # Computing the likelihood along the axis of interest for close relationship to fuse if needed.
            
            
            # Create a ball of confidence thresholds for the robot's point of collision with the threshold equal to corridor width
            # If less than threshold, plan to fuse them in future by setting a higher likelihood
            
        else: 
            l = 0
        
        # If less than the corridor width threshold, the likelihood is higher
        # If this is executed, create a flag to fuse the measurement and feature to get a bigger feature
        # Ultimately, a hyperplane or half-space is created, which forms a comaponent of planning for verification
        
        # If more, then initialize a new landmark
        
        return l
    
    def intialize_new_landmark(self,measurement,Qt_measurement_covariance):
        # Take care of the orientation, coordinate transformation from the robot frame to the world frame
        self.landmark_poses.append([self.pose[0],self.pose[1],self.pose[2]+atan(measurement)]) # Orientation update as FSM
        # We have to modify the orientation of the landmark to multiples of 90 degrees
        Hinv = np.array([[1,0,0],[0,1,0],[0,0,1/(1+measurement**2)]])
        Sigma = Hinv*Qt_measurement_covariance*
        self.landmark_covariances.append(Hinv*Sigma*Hinv.T) # Remember to add the covariance of the particle at that moment
        
    def update_landmark(self,measurment,landmark_number,Qt_measurement_covariance):
        # Updated landmark's estimated position and covariance using the correspondence variable
        # Fuse the landmark with measurements if the flag in the previous landmark_correspondence is True
        # After Fusion, revise the landmark set 
        # Define a hyperplane/halfspace to define a higher level feature set
        # The landmarks are defined as histograms rather than gaussian or particles
        pass
    
    def update_particle(self,measurement,number_of_landmarks,minimum_correspondence_likelihood,Qt_measurement_covariance):
        # This is the ultimate function for updating the particle filter
        
        # Compute the likelihood of the measurment to the list of number of landmarks - Correspondence calculation
        
        # If the likelihoods list is empty or the correspondence obtained for all the landmarks is less than the 
        # minimum_correspondence_likelihood, add a new landmark
        
        likelihoods = []
        if not likelihoods or max(likelihoods) < minimum_correspondence_likelihood:
            # Adding a new landmark
            return minimum_correspondence_likelihood
        
        # Else, update the landmark with the best likelihood correspondence and update thhe list to a higher feature set level,
        # which runs parallel by creating hyperplanes/halfspaces and planning thereby
        else:
            # Landmark update using EKF or UKF after computing the (max,argmax) of measurement_likelihoods
            # Find w, maximum likelihood and correspondence landmark index (indices for creating a hyperplane)
            
            return w

class FastSLAM:
    def __init__(self,intial_particles,robot_width,minimum_correspondence_likelihood,measurement_stddev,
                 control_speed_factor,control_head_factor, sample_time,robot_width):
        # Particles
        self.particles = intial_particles
        
        # Constants
        self.robot_width = robot_width
        self.minimum_correspondence_likelihood = minimum_correspondence_likelihood
        self.measurement_stddev = measurement_stddev
        self.control_speed_factor = control_speed_factor
        self.control_head_factor = control_head_factor
        self.dt = sample_time
        self.WB = robot_width
        
    def predict(self,control):
        # Prediction step of FastSLAM
        speed, head = control
        
        # Parabolic or 1D quadratic fit to the standard deviation of speed 
        speed_std = self.control_speed_factor * sqrt(speed) # To be modified
        head_std  = self.control_head_factor * sqrt(359 - head) # To be modified; mirror image of the speed deviation
        
        for p in self.particles:
            speed = gauss(speed,speed_std)
            head = gauss(head,head_std)
            p.move((speed,head),self.WB,self.dt)
    
    def update_and_compute_weights(self,landmarks):
        
        Qt_measurement_covariance = self.measurement_stddev ** 2 # Measurement covariance for the ratio
        
        weights = []
        
        for p in self.particles:
            
            number_of_landmarks = p.number_of_landmarks()
            weight = 1
            
            for measurement in landmarks:
                # Cumulative product of weights for a particle based on the measurements
                weight *= p.update_particle(measurement,number_of_landmarks,self.minimum_correspondence_likelihood,
                                            Qt_measurement_covariance)
                weights.append(weight)
                
            weights.append(weight)
            
        return weights
    
    def resample(self,weights):
        # Returns a new set of resampled particles, proportional to their weight
        new_particles = []
        max_weight = max(weights)
        index = randint(0, len(self.particles)-1)
        offset = 0.0
        for i in xrange(len(self.particles)):
            offset += uniform(0, 2.0 * max_weight)
            while offset > weights[index]:
                offset -= weights[index]
                index = (index + 1) % len(weights)
            new_particles.append(copy.deepcopy(self.particles[index]))
            
        return new_particles
        
    
    def correct(landmarks):
        # Correction step of FastSLAM
        weights = self.update_and_compute_weights(landmarks)
        self.particles = self.resample(weights)

In [6]:
import numpy as np
A = np.array([[1, 2],[3, 4]])
print A.T

[[1 3]
 [2 4]]
